In [16]:
import gensim
import pandas as pd

df = pd.read_json("reviews_Sports_and_Outdoors_5.json", lines=True)

In [17]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"
...,...,...,...,...,...,...,...,...,...
296332,A2XX2A4OJCDNLZ,B00LFPS0CY,RatherLiveInKeyWest,"[2, 3]",This is a water bottle done right. It is a ver...,5,Hydracentials Sporty 25 Oz Stainless Steel Wat...,1405036800,"07 11, 2014"
296333,A3LGT6UZL99IW1,B00LFPS0CY,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",If you're looking for an insulated water bottl...,5,"Large, incredibly well made water bottle!",1405641600,"07 18, 2014"
296334,ASKZO80Z1RKTR,B00LFPS0CY,Robin Lee,"[0, 0]","This Hydracentials Sporty 25 OZ, double insula...",5,"""Great Water Bottle For Hot Day""......",1405900800,"07 21, 2014"
296335,APRNS6DB68LLV,B00LFPS0CY,"Rob Slaven ""slavenrm@gmail. com""","[1, 1]",As usual I received this item free in exchange...,5,A pretty impressive water bottle. Best I've s...,1405900800,"07 21, 2014"


In [18]:
df.shape

(296337, 9)

# Simple Preprocessing & Tokenization
The first thing to do for any data science task is to clean the data. For NLP, we apply various processing like converting all the words to lower case, trimming spaces, removing punctuations. This is something we will do over here too.

Additionally, we can also remove stop words like 'and', 'or', 'is', 'the', 'a', 'an' and convert words to their root forms like 'running' to 'run'.

In [19]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [20]:
review_text[0]

['this',
 'came',
 'in',
 'on',
 'time',
 'and',
 'am',
 'veru',
 'happy',
 'with',
 'it',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 'very',
 'easy']

In [21]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [22]:
model.build_vocab(review_text, progress_per=1000)

In [23]:
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(91340146, 121496535)

In [24]:
model.wv.most_similar("awful")

[('horrible', 0.7131192088127136),
 ('terrible', 0.7049471139907837),
 ('ugly', 0.629944920539856),
 ('funny', 0.5801563262939453),
 ('horrendous', 0.5673969388008118),
 ('crappy', 0.5585536360740662),
 ('enormous', 0.5581580400466919),
 ('authentic', 0.5515291690826416),
 ('cheesy', 0.5497032999992371),
 ('unpleasant', 0.5467362403869629)]

In [25]:
import numpy as np

df["review_text"] = df.reviewText.apply(gensim.utils.simple_preprocess)

def get_embedding_w2v(doc_tokens):
    # print(a, doc_tokens)
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(100)
    else:
        for tok in doc_tokens:
            if tok in model.wv.vocab:
                embeddings.append(model.wv.word_vec(tok))
            else:
                embeddings.append(np.random.rand(100))

        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

In [26]:
df["vector"] = df["review_text"].apply(lambda x :get_embedding_w2v(x))

In [27]:
df["vector"].head(10)

0    [0.07443509, -0.07568184, -0.22597532, 0.15275...
1    [-0.46686304, 0.5730293, 0.31570125, -0.663524...
2    [0.15501946, 0.36357853, 1.09545, -0.121376716...
3    [-0.74412817, 0.15105678, 0.7857037, -1.209483...
4    [-0.40252483, 0.29291132, 0.50161403, -0.47091...
5    [0.072071396, 0.42543897, -0.5304567, -0.25088...
6    [0.052283697, 0.22927415, 1.1026174, -0.284596...
7    [0.16786967, 0.23844041, 0.24614152, 0.1690434...
8    [0.07474031, 0.5791853, -0.12612793, -0.131213...
9    [0.23585378, 0.51327395, 0.20040877, -0.283455...
Name: vector, dtype: object

In [37]:
from sklearn.metrics.pairwise import cosine_similarity
def ranking_ir(query):
  
  # pre-process Query
  query = gensim.utils.simple_preprocess(query)
  # generating vector
  vector=get_embedding_w2v(query)

  # ranking documents
  documents=df[['reviewText','summary']].copy()
  documents['similarity']=df['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents.sort_values(by='similarity',ascending=False,inplace=True)
  
  return documents.head(100).reset_index(drop=True)

In [38]:
ranking_ir("good product")

,reviewText,summary,similarity
0,Good product,Four Stars,1.000000
1,Good product,Five Stars,1.000000
2,good product!,Five Stars,1.000000
3,good product,Five Stars,1.000000
4,good product,Five Stars,1.000000
...,...,...,...
95,you cant beat it UTG sells some realy good pro...,very nice,0.694741
96,"good quality, good price.",Five Stars,0.694740
97,Excellent! Cheap! Item fit my Mossberg 500 p...,perfect,0.694642
98,Very well designed good quality product. Like ...,Nice product,0.694383


In [39]:
ranking_ir("it is costly")

,reviewText,summary,similarity
0,"This is worthless, And it is Worthless, It is ...",Only lasted 1week,0.851483
1,It is very functional but the trade off for th...,Was a gift,0.826292
2,"This is a good product, but it is a little wea...",Good product,0.822801
3,"Decent, it is what it is, and it works!",Tactical Keychains Approves!,0.810211
4,This is the second time I have ordered this ha...,Excellent value,0.802825
...,...,...,...
95,The tool is perfectly machined and works just ...,Well machined tool and does the job perfectly.,0.739789
96,"For those that know, it's a Fobus, nothing fan...",Good Value,0.739706
97,It is a Kettlebell. It weighs 30 pounds. The p...,What's Not To Like ?,0.739379
98,Its a whistle.. It is what it is but what I no...,Company produces great outdoor equipment,0.738661
